# Sentiment Analysis of COVID-19 Tweets: When did the Public Panic Set In?

    Notebook by Allison Kelly - allisonkelly42@gmail.com

# Introduction 

This notebook is part one of my NLP project aiming to scrape and analyze tweets regarding the coronavirus pandemic. View part two <a href ="https://github.com/akelly66/COVID-Tweet-Sentiment/blob/master/text-processing/COVID-tweet-NLP.ipynb">here</a>.

<b> More documentation to come.</b>

# Imports

In [2]:
import pandas as pd
import json
import requests

import searchtweets
from searchtweets import load_credentials
from string import Template

# Initial Request

In [11]:
credentials = load_credentials(filename="/Users/Allie/Documents/DS-Projects/API keys/twitter_keys.yaml",
                 yaml_key="search_tweets_api",
                 env_overwrite=False)

/Users/Allie/anaconda3/lib/python3.7/site-packages/searchtweets/credentials.py:34: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  search_creds = yaml.load(f)[yaml_key]
Grabbing bearer token from OAUTH


In [12]:
bearer_token = credentials['bearer_token']

In [7]:
endpoint = 'https://api.twitter.com/1.1/tweets/search/fullarchive/production.json'

headers = {"Authorization":"Bearer {}".format(bearer_token), "Content-Type": "application/json"}  

data = '{"query":"(coronavirus OR COVID-19)", "fromDate": "202001300000", "toDate": "202001312359"}, {"place":{"profile_country":"United States of America"}}'

response = requests.post(endpoint,data=data,headers=headers).json()

In [221]:
response['next']

'eyJtYXhJZCI6MTIyMzM5NTI1MzQ3NjExMDM0NH0='

In [181]:
print(json.dumps(response, indent = 2)).keys()

{
  "results": [
    {
      "created_at": "Fri Jan 31 23:58:59 +0000 2020",
      "id": 1223395284119695360,
      "id_str": "1223395284119695360",
      "text": "No les miento....\nEse Coronavirus me tiene asustada \ud83d\ude22\ud83d\ude22\ud83d\ude25",
      "source": "<a href=\"https://mobile.twitter.com\" rel=\"nofollow\">Twitter Web App</a>",
      "truncated": false,
      "in_reply_to_status_id": null,
      "in_reply_to_status_id_str": null,
      "in_reply_to_user_id": null,
      "in_reply_to_user_id_str": null,
      "in_reply_to_screen_name": null,
      "user": {
        "id": 1187527343562264576,
        "id_str": "1187527343562264576",
        "name": "\u2022| Wininini\ud83c\udf12\ud83d\udda4|\u2022",
        "screen_name": "winiffer_perez",
        "location": "Caracas Venezuela",
        "url": null,
        "description": "Aqu\u00ed vamos de nuevo...\nPerd\u00ed el perfil de @WinifferP_\nWe back to the 90's\ud83c\udf1a\u270c\ufe0f\nMaferef\u00fan Oy\u00e1 \ud83c\udf1

AttributeError: 'NoneType' object has no attribute 'keys'

In [171]:
covid_tweets = pd.DataFrame.from_dict(response['results'])

In [230]:
len(covid_tweets)

100

# Pagination

In [10]:
def paginate_covid_tweets(endpoint, headers, previous_response_token=None):
    
    global next_token
    
    if previous_response_token:
        t=Template('{"query":"(coronavirus OR COVID-19)", "fromDate": "202001300000", "toDate": "202001312359","next":"${next_token}"}, {"place":{"profile_country":"United States of America"}}')
        data = t.substitute(next_token=previous_response_token)
        
        response = requests.post(endpoint,data=data,headers=headers).json()
        next_token = response['next']
        
        return response, next_token
    
    elif previous_response_token == False: 
        
    
        data = '{"query":"(coronavirus OR COVID-19)", "fromDate": "202001300000", "toDate": "202001312359","next":"${next_token}"}, {"place":{"profile_country":"United States of America"}}'
    
    
        response = requests.post(endpoint,data=data,headers=headers).json()
        next_token = response['next']
        
        return response, next_token
    
    else:
        return "Response Error"

In [ ]:
next_token = response['next']
i = 0
dfs = []
while i < 51:
    response, next_token = paginate_covid_tweets(endpoint, headers, response['next'])
    dfs.append(response)
    i += 1

In [ ]:
for dictionary in dfs:
    df = pd.DataFrame.from_dict(dictionary['results'])
    covid_tweets = pd.concat([covid_tweets,df], axis=0)

In [20]:
# Final token
latest_token = dfs[-1]['next']

'eyJtYXhJZCI6MTIyMzM5NDQ0MzQwMDc2NTQ0MX0='

In [22]:
len(covid_tweets)

3700

# Next Steps